In [15]:
# Import Apache Beam library for building pipelines
import apache_beam as beam

# Import pipeline options and setup options for configuring the Beam pipeline
from apache_beam.options.pipeline_options import PipelineOptions, SetupOptions

# Import the MySQL connector library for interacting with MySQL databases
import mysql.connector

# Import the JSON library for parsing and manipulating JSON data
import json

# Import the regular expression (re) library for string pattern matching
import re

# Import the datetime module for handling dates and times
from datetime import datetime

# Import typing utilities for type hints
from typing import Dict, List, Optional

# Import dataclass for creating structured data classes
from dataclasses import dataclass

# Import logging module for logging messages
import logging

# Set up logging configuration with INFO level
logging.basicConfig(level=logging.INFO)

# Create a logger instance for logging messages
logger = logging.getLogger(__name__)

In [16]:

# SQL statement to create a metrics table in the database, if it doesn't already exist
CREATE_METRICS_TABLE = """
CREATE TABLE IF NOT EXISTS sms_metrics (
    id INT AUTO_INCREMENT PRIMARY KEY, -- Unique identifier for each record
    status VARCHAR(50), -- SMS status (e.g., sent, failed)
    total_cost DECIMAL(10,2), -- Total cost of messages in this status
    message_count INT, -- Total number of messages
    avg_cost DECIMAL(10,2), -- Average cost per message
    avg_word_count FLOAT, -- Average number of words per message
    most_common_hour INT, -- Hour when most messages were sent
    analysis_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP -- Timestamp of analysis
);
"""

In [17]:

class DatabaseWriter:
    """Handles database connections and writing results"""
    def __init__(self, config):
        self.config = config

    def setup_database(self):
        """Create the metrics table if it doesn't exist"""
        conn = mysql.connector.connect(**self.config)
        try:
            cursor = conn.cursor()
            cursor.execute(CREATE_METRICS_TABLE)
            conn.commit()
        finally:
            cursor.close()
            conn.close()

    def write_metrics(self, metrics):
        """Write metrics to database"""
        conn = mysql.connector.connect(**self.config)
        try:
            cursor = conn.cursor()
            insert_query = """
            INSERT INTO sms_metrics 
            (status, total_cost, message_count, avg_cost, avg_word_count, most_common_hour)
            VALUES (%(status)s, %(total_cost)s, %(message_count)s, 
                    %(avg_cost)s, %(avg_word_count)s, %(most_common_hour)s)
            """
            cursor.execute(insert_query, metrics)
            conn.commit()
            logger.info(f"Wrote metrics for status: {metrics['status']}")
        finally:
            cursor.close()
            conn.close()

In [18]:
@dataclass
class SMSRecord:
    """Data class for storing structured SMS data"""
    message_id: int
    status: str
    cost: float
    sender: str
    recipient: str
    message: str
    created_date: datetime
    blast_id: Optional[str]


In [19]:

class ReadFromMySQL(beam.DoFn):
    """Reads SMS records from MySQL database"""
    def __init__(self, connection_config: Dict[str, str], query: str, batch_size: int = 1000):
        self.connection_config = connection_config
        self.query = query
        self.batch_size = batch_size
        self.records_read = beam.metrics.Metrics.counter('main', 'records_read')

    def setup(self):
        self.connection = mysql.connector.connect(**self.connection_config)
        self.cursor = self.connection.cursor(buffered=True)

    def process(self, element) -> List[SMSRecord]:
        try:
            self.cursor.execute(self.query)
            while True:
                rows = self.cursor.fetchmany(self.batch_size)
                if not rows:
                    break
                for row in rows:
                    self.records_read.inc()
                    yield self._convert_to_sms_record(row)
        except Exception as e:
            logger.error(f"Error reading from database: {e}")
            raise

    @staticmethod
    def _parse_datetime(date_str):
        if isinstance(date_str, datetime):
            return date_str
        try:
            for fmt in ('%Y-%m-%d %H:%M:%S', '%Y-%m-%d %H:%M:%S.%f'):
                try:
                    return datetime.strptime(str(date_str), fmt)
                except ValueError:
                    continue
            return datetime.now()
        except Exception as e:
            logger.warning(f"Date parsing failed for {date_str}: {e}")
            return datetime.now()

    @staticmethod
    def _convert_to_sms_record(row) -> SMSRecord:
        id, message, sender, recipient, _, _, response, created_dt, _, _, status, blast_id, _ = row
        
        cost = 0.0
        try:
            response_data = json.loads(response)
            cost = float(response_data["Recipients"][0]["cost"].replace("KES ", ""))
        except (json.JSONDecodeError, KeyError, IndexError):
            logger.warning(f"Failed to parse cost for message {id}")

        return SMSRecord(
            message_id=id,
            status=status,
            cost=cost,
            sender=sender,
            recipient=recipient,
            message=message,
            created_date=ReadFromMySQL._parse_datetime(created_dt),
            blast_id=blast_id
        )


In [20]:

class SMSAnalyzer(beam.DoFn):
    """Analyzes SMS data for various metrics"""
    def process(self, record: SMSRecord):
        word_count = len(self._tokenize_message(record.message))
        
        analysis = {
            'status': record.status,
            'cost': record.cost,
            'word_count': word_count,
            'hour_of_day': record.created_date.hour if record.created_date else 0,
            'blast_id': record.blast_id
        }
        
        yield analysis

    @staticmethod
    def _tokenize_message(message: str) -> List[str]:
        return re.findall(r'\w+', message.lower())

In [21]:


def run_pipeline(connection_config: Dict[str, str], query: str):
    """Main pipeline execution function"""
    
    # Initialize database writer and create table
    db_writer = DatabaseWriter(connection_config)#A class instance up top
    db_writer.setup_database()
    
    pipeline_options = PipelineOptions()
    #is used in Apache Beam pipelines to ensure that the main Python session
    #(where your code is defined) is serialized and accessible across all workers in a distributed environment. Google Dataflow
    pipeline_options.view_as(SetupOptions).save_main_session = True

    with beam.Pipeline(options=pipeline_options) as pipeline:
        analysis_results = (
            pipeline
            | 'Create Initial' >> beam.Create([None])#No initial data is passed to the pipeline
            | 'Read SMS Data' >> beam.ParDo(ReadFromMySQL(connection_config, query)) # Pull data from the db
            | 'Analyze SMS' >> beam.ParDo(SMSAnalyzer())
        )

        # Calculate and save metrics
        def save_metrics(metrics):
            """Save metrics to database and print results"""
            formatted_metrics = {
                'status': metrics['status'],
                'total_cost': metrics['total_cost'],
                'message_count': metrics['message_count'],
                'avg_cost': metrics['avg_cost'],
                'avg_word_count': metrics['avg_word_count'],
                'most_common_hour': metrics['most_common_hour']
            }
            
            # Write to database
            db_writer.write_metrics(formatted_metrics)
            
            # Print results
            print(f"\nStatus: {metrics['status']}")
            print(f"Total Cost: KES {metrics['total_cost']:.2f}")
            print(f"Message Count: {metrics['message_count']}")
            print(f"Average Cost per Message: KES {metrics['avg_cost']:.2f}")
            print(f"Average Word Count: {metrics['avg_word_count']:.1f}")
            print(f"Most Common Hour: {metrics['most_common_hour']:02d}:00")
            print("-" * 50)

        _ = (
            analysis_results
            | 'Key By Status' >> beam.Map(lambda x: (x['status'], x))#Groups data by the status field.
            | 'Group By Status' >> beam.GroupByKey()#Groups all records with the same status into a single key-value pair.
            | 'Calculate Status Metrics' >> beam.Map(# Calculates metrics for each status group.
                lambda x: {
                    'status': x[0],
                    'total_cost': sum(item['cost'] for item in x[1]),
                    'message_count': len(list(x[1])),
                    'avg_cost': sum(item['cost'] for item in x[1]) / len(list(x[1])),
                    'avg_word_count': sum(item['word_count'] for item in x[1]) / len(list(x[1])),
                    'most_common_hour': max(
                        set(item['hour_of_day'] for item in x[1]),
                        key=lambda h: sum(1 for item in x[1] if item['hour_of_day'] == h)
                    )
                }
            )
            | 'Save Results' >> beam.Map(save_metrics)#Saves the calculated metrics and prints the output.
        )

if __name__ == '__main__':
    connection_config = {
        'host': '127.0.0.1',
        'database': 'defaultdb',
        'user': 'root',
        'password': 'cypher'
    }
    
    query = 'SELECT * FROM smslog LIMIT 20000;'#this can be changed to adapt to the requirements.
    run_pipeline(connection_config, query)

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:apache_beam.runners.worker.statecache:Creating state cache with size 104857600
INFO:__main__:Wrote metrics for status: 
INFO:__main__:Wrote metrics for status: Success
INFO:__main__:Wrote metrics for status: Failed
INFO:__main__:Wrote metrics for status: Expired



Status: 
Total Cost: KES 7770.40
Message Count: 13045
Average Cost per Message: KES 0.60
Average Word Count: 32.0
Most Common Hour: 08:00
--------------------------------------------------

Status: Success
Total Cost: KES 3796.00
Message Count: 6637
Average Cost per Message: KES 0.57
Average Word Count: 28.9
Most Common Hour: 07:00
--------------------------------------------------

Status: Failed
Total Cost: KES 180.00
Message Count: 291
Average Cost per Message: KES 0.62
Average Word Count: 33.9
Most Common Hour: 07:00
--------------------------------------------------

Status: Expired
Total Cost: KES 27.00
Message Count: 27
Average Cost per Message: KES 1.00
Average Word Count: 37.7
Most Common Hour: 08:00
--------------------------------------------------
